# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Generate IQ data

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 11
ROUNDS = 11
OTHER_DATE = '2021-11-11'

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


In [4]:
from Scratch import create_or_load_kde_grid

grid_dict, processed_scaler_dict = create_or_load_kde_grid(provider, tobecalib_backend=DEVICE, 
                                                           num_grid_points=300, num_std_dev=2, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


In [121]:
# Get noise model
p1Q = 0
pZ = 0
pRE = 0
pXY = 0

p2Q = 0.1e-2
pRO = 15e-2

p1Q = 1e-4
p2Q = 6e-3
pXY = 1e-4
pZ = 1e-4
pRO = 1e-1
pRE = 0

noise_model = simulator.get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE)

In [122]:
SHOTS = int(1e4)
LOGICAL = 0
IQ_data = simulator.generate_IQ(SHOTS, noise_model=noise_model, logical=LOGICAL)

19:09:44 Warning: Getting counts via stim. This may take time.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/IQ_data/simulator.py, LINE: 43


100%|██████████| 10000/10000 [00:00<00:00, 20501.05it/s]


# Decode

In [123]:
import pymatching
import stim


circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=p2Q, #two-qubit-fidelity,
                                after_reset_flip_probability=pRE, #reset error,
                                before_measure_flip_probability=pRO, #measurement error,
                                before_round_data_depolarization=pXY + pZ) #idle error)
# print(circuit)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)

In [124]:
import cpp_soft_info

matching = pymatching.Matching.from_detector_error_model(model)
# cpp_soft_info.reweight_edges_based_on_error_probs(matching._matching_graph, counts, False, "spitz")

p_data = 6.869e-1 # mean sherbrooke noise
num_errors = cpp_soft_info.decode_IQ_shots(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), simulator.qubit_mapping, grid_dict,
                                           processed_scaler_dict, p_data=-1, p_mixed=-1, #p_mixed=1e-80, for d=30
                                           common_measure=-1, _bimodal=False, merge_strategy = "smallest-weight")
print("num_errors:", num_errors, "out of", len(IQ_data), "shots")

num_errors: 29 out of 10000 shots


In [125]:
matching = pymatching.Matching.from_detector_error_model(model)
num_errors = cpp_soft_info.decode_IQ_shots_flat(matching._matching_graph, IQ_data,
                                           ROUNDS, int(LOGICAL), simulator.qubit_mapping, grid_dict,
                                           processed_scaler_dict)
print("num_errors:", num_errors, "out of", len(IQ_data), "shots")


num_errors: 139 out of 10000 shots


In [127]:
matching = pymatching.Matching.from_detector_error_model(model)

import numpy as np
def weight_to_prob(weight):
    return 1/(1+np.exp(weight))

p_data = 1.869e-3# mean sherbrooke ECR error
p_mixed = p_data# Same as weighted
# p_meas = 1e-3
p_meas = 15.900e-2 # random found number


p_data = -1
p_mixed = -1
p_meas = -1

num_errors = cpp_soft_info.decode_IQ_shots_flat_informed(matching._matching_graph, IQ_data, 
                                           ROUNDS, int(LOGICAL), simulator.qubit_mapping, grid_dict, processed_scaler_dict,
                                           p_data = p_data, p_mixed = p_mixed, p_meas = p_meas, common_measure=-1)

print("num_errors:", num_errors, "out of", len(IQ_data), "shots")
         
# takes 1s

num_errors: 13 out of 10000 shots
